In [1]:
import requests
from bs4 import BeautifulSoup
import copy
import yaml
import json
import os
import openai
import tiktoken
import pandas as pd

URL = 'https://www.psp.cz/eknih/2021ps/stenprot/index.htm'  # 2021
# URL = 'https://www.psp.cz/eknih/2017ps/stenprot/index.htm'  # 2017
# URL = 'https://www.psp.cz/eknih/2013ps/stenprot/index.htm'  # 2013

key_word = "Andrej Babiš"
important_words = ["předsed", "poslan", "ministr", "Předsed", "Poslan", "Ministr"]

In [61]:
# Required Libraries

# Constants

def fetch_website_content(url):
    """
    Fetches the content of the website at the given URL.
    
    Args:
        url (str): The target URL to fetch.
    
    Returns:
        str: The raw content of the webpage.
    """
    try:
        response = requests.get(url)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
    except:
        print("Scrappe URL unsucessful: ", url)
    # Check if the request was successful
    # if response.status_code != 200:
    #     raise ValueError(f"Failed to fetch the webpage. HTTP Status Code: {response.status_code}")
    return soup

def extract_html_of_a_with_href(soup):
    """
    Parses the required data from the given HTML content.
    
    Args:
        html_content (str): The raw HTML content.
    
    Returns:
        list: A list of data extracted from the content.
    """
    # Modify this section to extract the specific data you need
    # For instance, to extract all the text inside paragraph tags:
    data = [(a.get_text(), a['href']) for a in soup.find_all('a', href=True)]
    return data

def extract_html_of_div_with_id(soup, target_id):
    """
    Extracts the inner HTML content of a div with the given ID from the provided HTML content.
    
    Args:
        html_content (str): The raw HTML content.
        target_id (str): The ID of the div whose content needs to be extracted.
    
    Returns:
        str: The inner HTML content of the div, or None if the div isn't found.
    """
    # Find the div with the given ID
    div = soup.find('div', id=target_id)
    
    # Return its inner HTML if found
    return div if div else None

def extract_content_of_p_with_align(html_content, target_align):
    """
    Extracts the content of all <p> tags with the specified align attribute.
    
    Args:
        html_content (str): The raw HTML content.
        target_align (str): The align attribute value to look for.
    
    Returns:
        list: A list of content extracted from the <p> tags.
    """
    # Find all <p> tags with the specified align attribute
    p_tags = html_content.find_all('p', align=target_align)
    
    # Extract and return the content of these tags
    return [p.get_text() for p in p_tags]

def extract_content_of_a_with_class(soup, target_class):
    """
    Extracts the content of all <a> tags with the specified class attribute.
    
    Args:
        html_content (str): The raw HTML content.
        target_class (str): The class attribute value to look for.
    
    Returns:
        list: A list of content extracted from the <a> tags.
    """
    # Find all <a> tags with the specified class attribute
    a_tags = soup.find_all('a', class_=target_class)
    
    # Extract and return the content of these tags
    return [a['href'] for a in a_tags]

def find_a_tags_in_p(soup, target_align):
    """
    Finds all <a> tags that are nested within <p> tags.
    
    Args:
        html_content (str): The raw HTML content.
    
    Returns:
        list: A list of all <a> tags found within <p> tags.
    """
    # Find all <p> tags
    p_tags = soup.find_all('p', align=target_align)
    
    # For each <p> tag found, find all nested <a> tags
    a_tags_within_p = [a.get_text() for p in p_tags for a in p.find_all('a')]
    
    return a_tags_within_p

def replace_special_characters(text):
    return text.replace("ì", "ě").replace("\x9e", "ž").replace("\x9a", "š").replace("\x8a", "Š").replace("ø","ř").replace("\xd8", "Ř").replace("è","č").replace("È", "Č").replace("\xa0", " ").replace("\xf9", "ů").replace("\xf2", "ň").replace("\xef", "ď").replace("\n", "").replace("x9d", "ť").replace("x8e", "Ž")

In [3]:
def get_all_schuze_links(URL):
    url_data = {}
    soup = fetch_website_content(URL)
    body_content = extract_html_of_div_with_id(soup, "body")
    hrefs = extract_html_of_a_with_href(body_content)
    links = [link for link in hrefs if "schuz" in link[1] and "index" in link[1]]
    for link in links:
        url_data[replace_special_characters(link[0])] = {"url": URL.split("/index")[0] + "/" + link[1]}

    return url_data

def get_single_schuze_content(URL_schuze):
    soup = fetch_website_content(URL_schuze)
    body_content = extract_html_of_div_with_id(soup, "main-content")
    hrefs = extract_html_of_a_with_href(body_content)
    links = [link[1] for link in hrefs if "-" in link[1]]
    single_url_schuze = URL_schuze.split("/index")[0] + "/" + links[0]

    return single_url_schuze

def get_single_speaker(URL_schuze, URL_zaznam):
    soup = fetch_website_content(URL_zaznam)
    body_content = extract_html_of_div_with_id(soup, "body")
    hrefs = extract_html_of_a_with_href(body_content)
    links = [link for link in hrefs if "#r" in link[1]]
    # url_data = URL_schuze.split("/index")[0] + "/" + links[0][1]
    return links[0][1]

def get_all_content_url(single_speaker, url_base):
    url_next = [single_speaker]
    all_content = []
    while(len(url_next) != 0):
        single_link = url_base + url_next[0]
        all_content += [single_link]
        soup = fetch_website_content(single_link)
        url_next = extract_content_of_a_with_class(soup, "next")
    
    return all_content

In [4]:
schuze_all = get_all_schuze_links(URL)
print(schuze_all)

{'1. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/001schuz/index.htm'}, '2. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/002schuz/index.htm'}, '3. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/003schuz/index.htm'}, '4. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/004schuz/index.htm'}, '5. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/005schuz/index.htm'}, '6. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/006schuz/index.htm'}, '7. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/007schuz/index.htm'}, '8. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/008schuz/index.htm'}, '9. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/009schuz/index.htm'}, '10. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/010schuz/index.htm'}, '11. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenprot/011schuz/index.htm'}, '12. schůze': {'url': 'https://www.psp.cz/eknih/2021ps/stenpro

In [34]:
def get_key_tags(all_speakers):
    important_urls = []

    for key in all_speakers:
        for element in all_speakers[key]:
            if(key_word in element):
                important_urls += [key]
    
    speaker_found = False
    
    for key in all_speakers:
        if(key in important_urls):
            speaker_found = True
        else:
            if(speaker_found):
                if(len(all_speakers[key]) == 0):
                    important_urls += [key]
                else:
                    important_urls += [key]
                    speaker_found = False

    p_tags = []
    for imp_url in important_urls:
        p_tags += fetch_website_content(imp_url).find_all('p', align="justify")
    
    key_speaker = False
    useful_tags = []
    
    for p in p_tags:
        a_tags = p.find_all('a')
        if(len(a_tags) != 0):
            speaker_name = replace_special_characters(a_tags[0].get_text())
            if(key_word in speaker_name):
                useful_tags += [replace_special_characters(p.get_text())]
                key_speaker = True
            else:
                key_speaker = False
        else:
            if(key_speaker):
                useful_tags += [replace_special_characters(p.get_text())]
    
    return list(set(useful_tags))

In [10]:
def all_links(schuze):
    url_base = schuze["url"].split("index")[0]
    single_content = get_single_schuze_content(schuze["url"])
    single_speaker = get_single_speaker(schuze["url"], single_content)
    all_content = get_all_content_url(single_speaker, url_base)

    all_speakers = {}
    
    for url_single_content in all_content:
        soup_link = fetch_website_content(url_single_content)
        body_content_link = extract_html_of_div_with_id(soup_link, "body")
        a_alings_link = find_a_tags_in_p(body_content_link, "justify")
        all_speakers[url_single_content] = [replace_special_characters(tag) for tag in a_alings_link]
    
    return get_key_tags(all_speakers)

In [35]:
significant_content = {}

for key in schuze_all:
    try:
        significant_content[key] = all_links(schuze_all[key])
        print("Done downloading: ", key)
    except:
        print("Downloading: ", key, " unsucessful!")

Done donwloading:  1. schůze
Done donwloading:  2. schůze
Done donwloading:  3. schůze
Done donwloading:  4. schůze
Done donwloading:  5. schůze
Done donwloading:  6. schůze
Done donwloading:  7. schůze
Done donwloading:  8. schůze
Done donwloading:  9. schůze
Done donwloading:  10. schůze
Done donwloading:  11. schůze
Done donwloading:  12. schůze
Done donwloading:  13. schůze
Done donwloading:  14. schůze
Done donwloading:  15. schůze
Done donwloading:  16. schůze
Done donwloading:  17. schůze
Done donwloading:  18. schůze
Done donwloading:  19. schůze
Done donwloading:  20. schůze
Done donwloading:  21. schůze
Done donwloading:  22. schůze
Done donwloading:  23. schůze
Done donwloading:  24. schůze
Done donwloading:  25. schůze
Done donwloading:  26. schůze
Done donwloading:  27. schůze
Done donwloading:  28. schůze
Done donwloading:  29. schůze
Done donwloading:  30. schůze
Done donwloading:  31. schůze
Done donwloading:  32. schůze
Done donwloading:  33. schůze
Done donwloading:  

In [36]:
# len(significant_content["71. schůze"])

In [37]:
# len(list(set(significant_content["71. schůze"])))

In [38]:
# list(set(significant_content["71. schůze"]))

In [39]:
file_name = "./data/PS2021_data.yaml"

with open(file_name, "w", encoding="utf-8") as file:
    yaml.dump(significant_content, file, allow_unicode=True)

In [40]:
output_file = "./data/PS2021_data.json"

with open(output_file, 'w', encoding="utf-8") as outfile:
    json.dump(significant_content, outfile, ensure_ascii=False, indent=2)

In [41]:
base_prompt = "Assume you are a software developer who is involved in LLM design. I am building an LLM model to simulate a particular person. I have a speech by a person who speaks on a certain topic in that speech. I need to prepare this speech to go as input to a fine-tuning openai. By making it only a speech, I need to rewrite this speech to be a form of dialogue so that the 'assistant' asks questions and the said person answers them. I would also need to remove the editorial notes (listed in brackets) in the speech. Could you please rewrite the following speech for me in this way. Please rewrite it in a dialogue form where the 'assistant' asks multiple questions and the person, i.e. 'user', answers them. The context of the system is: 'You are Andrej Babiš, Czech politician and businessman.'. Write the output in a code window so that I can copy it, put it in a format that can be used for fine-tuning openai and do not translate the text, instead leave it in Czech. Example: {'messages': [{'role': 'system', 'content': 'Jsi Andrej Babiš, Český politik, bývalý premiér a podnikatel.'}, {'role': 'user', 'content': 'Prezident Pavel, premiér Fiala, předsedkyně Poslanecké sněmovny Pekarová Adamová, ani předseda Senátu Vystrčil skutečně Robertu Ficovi k jeho vítězství ve slovenských parlamentních volbách negratulovali.'}, {'role': 'assistant', 'content': 'Nikdo, ani prezident, ani premiér, ani předsedkyně Sněmovny, ani předseda Senátu, Ficovi negratulovali.'}]} \n\nText to be preprocessed:"

In [42]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
base_len = len(encoding.encode(base_prompt))
print(base_len)

418


In [43]:
sum = 0
df_tokens = pd.DataFrame({"schuze": [], "content": [], "tokens": []})

for key in significant_content:
    print("Number of tokens in ", key, " calculated.")
    for element in significant_content[key]:
        element_len = len(encoding.encode(element))
        df_tokens = df_tokens._append({"schuze": key, "content": element, "tokens": element_len}, ignore_index = True)

df_tokens.describe()

Number of tokens in  1. schůze  calculated.
Number of tokens in  2. schůze  calculated.
Number of tokens in  3. schůze  calculated.
Number of tokens in  4. schůze  calculated.
Number of tokens in  5. schůze  calculated.
Number of tokens in  6. schůze  calculated.
Number of tokens in  7. schůze  calculated.
Number of tokens in  8. schůze  calculated.
Number of tokens in  9. schůze  calculated.
Number of tokens in  10. schůze  calculated.
Number of tokens in  11. schůze  calculated.
Number of tokens in  12. schůze  calculated.
Number of tokens in  13. schůze  calculated.
Number of tokens in  14. schůze  calculated.
Number of tokens in  15. schůze  calculated.
Number of tokens in  16. schůze  calculated.
Number of tokens in  17. schůze  calculated.
Number of tokens in  18. schůze  calculated.
Number of tokens in  19. schůze  calculated.
Number of tokens in  20. schůze  calculated.
Number of tokens in  21. schůze  calculated.
Number of tokens in  22. schůze  calculated.
Number of tokens in

,tokens
count,1929.000000
mean,311.299637
std,198.990057
min,1.000000
25%,178.000000
50%,280.000000
75%,400.000000
max,1727.000000


In [44]:
df_tokens.to_csv('./data/PS2021_tokens.csv', header = True, sep = ';')

In [45]:
df_tokens.tokens.sum()

600497.0

In [67]:
schuze_active = "3. schůze"

df_tokens[df_tokens.schuze == schuze_active]

,schuze,content,tokens
0,3. schůze,"""Vyhláškou"" - a tady čtu podklad od pana minis...",252.0
1,3. schůze,Vítám návrh pana předsedy Okamury. Pro nás to ...,300.0
2,3. schůze,Takže hospitalizace je... tam je vidět jasný e...,338.0
3,3. schůze,Covid-19 mnohem vážnějšími zdravotními dopady ...,199.0
4,3. schůze,"Nebo maminky, těhotné ženy, ano. Když se zeptá...",175.0
5,3. schůze,"Není to rozhodnutí vlády, ale my jsme o tom sa...",209.0
6,3. schůze,"Myslím, že... A já nemám problém, nevím, jak t...",245.0
7,3. schůze,"Předseda vlády ČR Andrej Babiš: Dobrý den, váž...",98.0
8,3. schůze,V přepočtu na 100 000 obyvatel je v listopadu ...,405.0
9,3. schůze,A co se změnilo? Minulý rok? Minulý rok všichn...,373.0


In [54]:
names_active = list(set(df_tokens.schuze.values))

In [55]:
for schuze_active in names_active:
    print(schuze_active, "total tokens: ", df_tokens[df_tokens.schuze == schuze_active].tokens.sum())

20. schůze total tokens:  54671.0
72. schůze total tokens:  3604.0
41. schůze total tokens:  1294.0
76. schůze total tokens:  38841.0
11. schůze total tokens:  4554.0
68. schůze total tokens:  13545.0
62. schůze total tokens:  8663.0
35. schůze total tokens:  25305.0
33. schůze total tokens:  9866.0
4. schůze total tokens:  5993.0
75. schůze total tokens:  19703.0
23. schůze total tokens:  8804.0
77. schůze total tokens:  8650.0
78. schůze total tokens:  20896.0
6. schůze total tokens:  30951.0
30. schůze total tokens:  6881.0
79. schůze total tokens:  38300.0
36. schůze total tokens:  7073.0
70. schůze total tokens:  26408.0
67. schůze total tokens:  63461.0
19. schůze total tokens:  2866.0
9. schůze total tokens:  4482.0
54. schůze total tokens:  9487.0
48. schůze total tokens:  26002.0
65. schůze total tokens:  27971.0
3. schůze total tokens:  6108.0
60. schůze total tokens:  7843.0
63. schůze total tokens:  12353.0
71. schůze total tokens:  46207.0
7. schůze total tokens:  3718.0
1

In [66]:
df_tokens

,schuze,content,tokens
0,3. schůze,"""Vyhláškou"" - a tady čtu podklad od pana minis...",252.0
1,3. schůze,Vítám návrh pana předsedy Okamury. Pro nás to ...,300.0
2,3. schůze,Takže hospitalizace je... tam je vidět jasný e...,338.0
3,3. schůze,Covid-19 mnohem vážnějšími zdravotními dopady ...,199.0
4,3. schůze,"Nebo maminky, těhotné ženy, ano. Když se zeptá...",175.0
...,...,...,...
1924,79. schůze,Ale za vlády pana premiéra Fialy je zdravotnic...,454.0
1925,79. schůze,Takže závěrem. Pan premiér jako správný propag...,290.0
1926,79. schůze,"Vy hlavně byste měli skončit proto, že neplnít...",381.0
1927,79. schůze,Tohle dokonce musel psát i ten provládní aktiv...,80.0


In [74]:
dict_prompts = {}
max_tokens = 4096

for schuze_active in names_active:
    dict_prompts[schuze_active] = []
    df_active = df_tokens[df_tokens.schuze == schuze_active]
    tokens_sum = base_len
    messages_sum = base_prompt
    for index, row in df_active.iterrows():
        if((tokens_sum + row.tokens) < max_tokens):
            messages_sum += " "
            messages_sum += row.content
            tokens_sum += row.tokens
        else:
            dict_prompts[schuze_active] += [messages_sum]
            messages_sum = base_prompt + " " + row.content
            tokens_sum = base_len + row.tokens

print(dict_prompts)

{'20. schůze': ["Assume you are a software developer who is involved in LLM design. I am building an LLM model to simulate a particular person. I have a speech by a person who speaks on a certain topic in that speech. I need to prepare this speech to go as input to a fine-tuning openai. By making it only a speech, I need to rewrite this speech to be a form of dialogue so that the 'assistant' asks questions and the said person answers them. I would also need to remove the editorial notes (listed in brackets) in the speech. Could you please rewrite the following speech for me in this way. Please rewrite it in a dialogue form where the 'assistant' asks multiple questions and the person, i.e. 'user', answers them. The context of the system is: 'You are Andrej Babiš, Czech politician and businessman.'. Write the output in a code window so that I can copy it, put it in a format that can be used for fine-tuning openai and do not translate the text, instead leave it in Czech. Example: {'messag

In [77]:
file_name = "./data/PS2021_prompts.yaml"

with open(file_name, "w", encoding="utf-8") as file:
    yaml.dump(dict_prompts, file, allow_unicode=True)

In [ ]:
df_prompts = pd.DataFrame({"schuze": {}})

for key in significant_content:
    print("Number of tokens in ", key, " calculated.")
    for element in significant_content[key]:
        element_len = len(encoding.encode(element))
        df_tokens = df_tokens._append({"schuze": key, "content": element, "tokens": element_len}, ignore_index = True)

df_tokens.describe()

In [47]:
send_data_bool = input("Send data to openai? [yes/no]: ")

if(send_data_bool == "yes"):
    # Set up your API key from OpenAI
    openai.api_key = 'your-api-key'
    
    def send_prompt(prompt):
        response = openai.Completion.create(
          engine="gpt-3.5-turbo",  # or another model
          prompt=prompt,
          max_tokens=150
        )
        return response.choices[0].text.strip()
    
    # Example usage
    prompts = ["What is the capital of France?", "Explain the theory of relativity"]
    
    # Where to save the outputs
    output_file = "chatgpt_responses.txt"
    
    with open(output_file, "w") as file:
        for p in prompts:
            response = send_prompt(p)
            file.write(f"Prompt: {p}\nResponse: {response}\n\n")
else:
    print("Save your money ;)")

Send data to openai? [yes/no]:  nope


Save your money ;)


In [48]:
# reformated_data = {}

# for cont in significant_content:
#     reformated_data[replace_special_characters(cont)] = []
#     for elem in significant_content[cont]:
#         reformated_data[replace_special_characters(cont)] += [replace_special_characters(elem)]

# print(reformated_data)

In [49]:
# for cont in significant_content:
#     for elem in significant_content[cont]:
#         if("\x8a" in elem):
#             print(elem)
#             print("\n")